In [2]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time, os

chromedriver = "C:/Users/tanpe/Downloads/chromedriver_win32/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)

In [3]:
driver = webdriver.Chrome(chromedriver)
driver.get('https://factba.se/transcripts/speeches')

In [4]:
for i in range(15):
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);" )
    time.sleep(2)

soup = BeautifulSoup(driver.page_source, 'html.parser')

all_rallies = []

for tag in soup.find_all('h3'):
    if 'Rally' in tag.findNext().text and int(tag.findNext()['href'][-4:])>=2017:
        webpage = 'https://factba.se/'+str(tag.findNext()['href'])
        response = requests.get(webpage)
        subpage = response.text
        subsoup = BeautifulSoup(subpage,'lxml')

        title = subsoup.find('h1').text
        place = title.split('-')[0]
        date = title.split('-')[-1].strip().replace(',','')
        
        transcript_list = []
        
        for seq in subsoup.find_all('div', class_="transcript-text-block"):
            transcript_list.append(seq.findNext().text)
        
        transcript = ' '.join(transcript_list)
        
        rally = {'Place':place,'Date':date,'Transcript':transcript}
        
        all_rallies.append(rally)

In [47]:
all_rallies_df = pd.DataFrame(all_rallies)

In [48]:
all_rallies_df = all_rallies_df[8:12]
all_rallies_df

,Place,Date,Transcript
8,Speech: Donald Trump Holds a Campaign Rally in...,October 16 2020,"I love Georgia here, do I love being with you...."
9,Speech: Donald Trump Holds a Campaign Rally in...,October 15 2020,"Ah, thank you very much. Thank you let's see i..."
10,Speech: Donald Trump Holds a Campaign Rally in...,October 14 2020,"[Audience chants ""USA""] Thank you. Thank you v..."
11,Speech: Donald Trump Holds a Campaign Rally in...,October 13 2020,"Hello, Johnstown. Thank you very much. This is..."


In [49]:
all_rallies_df.to_csv('Trump_Rallies_Test_Set.csv')